# Adicionando requisitos

Nesta primeira parte do script, são importadas as bibliotecas necessárias e carregadas as variáveis de ambiente com load_dotenv(), isolando do codigo dados, como a chave a api do Groq. Em seguida, a biblioteca groq é importada, necessária para instanciar o cliente e permitir o uso do modelo de IA.

In [9]:
from dotenv import load_dotenv
import os
from groq import Groq

load_dotenv()

True

# Criação de client

Com a chave da API carregada nas variáveis de ambiente, é feito um teste para verificar a resposta do modelo a uma pergunta. Esse passo também ajuda a entender a estrutura dos objetos retornados pela API.

In [4]:
client = Groq(api_key=os.environ.get("API_GROQ"))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"} # pergunta realizada
    ],
    model="llama3-70b-8192", # modelo escolhido
    temperature=0 # parametro para gerar respostas mais diretas e previsíveis
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Inference**: Fast language models can perform inference (i.e., make predictions) quickly, which is essential for applications that require rapid decision-making, such as sentiment analysis, entity recognition, and question answering.
3. **Scalability**: Fast language models can handle large volumes of data and scale to meet the demands of big data and high-traffic applications. This is critical in industries like customer service, where handling a large volume of conversations simultaneously is essentia

# Criação da Classe Agent

Nesta parte do código, é criada manualmente a classe Agent, responsável por estruturar o agente de IA. Ela possui dois métodos especiais, "__init__" que inicializa o agente, definindo o cliente, mensagens e a instrução de sistema e o "__call__" permitindo que o agente seja chamado como uma função, registrando a entrada do usuário e retornando a resposta, além disso, há o método execute (não especial), que envia as mensagens para o modelo via API e retorna o conteúdo da resposta.

In [5]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None: # client será o groq e system será o prompt criado 
        self.client = client
        self.system = system
        self.messages: list = [] # historico de inputs e outputs do usuário e do modelo em suas intereções
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""): # junto ao metodo __init__ esses objetos formam a memória do modelo 
        if message:
            self.messages.append({"role": "user", "content": message}) # adiciona as informações á lista de mensagens
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result}) # adiciona as informações á lista de mensagens
        return result

    def execute(self): # chamada ao modelo 
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content # objeto que armazena a resposta do modelo

# Prompt e Tools

Nesta parte do código, é definido o system_prompt, que orienta o comportamento do agente durante a execução. O prompt descreve o ciclo de pensamento e ação no padrão de design react, lista as funções disponíveis e exemplifica como e quando utilizá-las, também são implementadas as ferramentas do agente:

calculate: realiza operações matemáticas usando Python.

get_planet_mass: retorna a massa de um planeta específico em quilogramas.

Combinadas ao system_prompt e a uma estrutura de repetição, essas funções permitem que o agente responda de forma autônoma e personalizada.

In [6]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


def calculate(operation: str) -> float: # função de calculo
    return eval(operation)


def get_planet_mass(planet) -> float: # simula um banco de dados, armazenando os valores das massas dos planetas
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

# Instanciando o Agente

Nesta etapa, o agente é instanciado utilizando o client configurado anteriormente e o system_prompt definido nas células anteriores. Isso cria um objeto já preparado para executar interações seguindo as instruções e ferramentas especificadas no prompt orientador.

In [7]:
neil_tyson = Agent(client=client, system=system_prompt)

# Loop de repetição

Nesta parte, é criada a função loop, responsável por automatizar o raciocínio do modelo, a função recebe uma pergunta inicial e executa interações com o agente até atingir o número máximo definido por max_iterations, evitando loops infinitos, durante cada iteração, o resultado do agente é analisado e caso contenha uma ação com pausa, a função identifica a tool solicitada, executa-a e envia a observação resultante de volta ao agente, as etapas são exibidas no console para acompanhar o fluxo de execução atravez dos prints. 

Essa estrutura garante que o agente siga, de forma controlada, o ciclo Pensamento - Ação - PAUSA - Observação, utilizando as ferramentas necessárias (calculate e get_planet_mass).

In [8]:
import re


def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt) # agente instanciado

    tools = ["calculate", "get_planet_mass"] # lista de ferramentas

    next_prompt = query # iniciar variavel next_prompt com entrada de usuário

    i = 0 # contador de iterações
  
    while i < max_iterations: # inicio do loop, com limite de iterações = 10
        i += 1
        result = agent(next_prompt)
        print(result) # Início de

        if "PAUSE" in result and "Action" in result: # verificação da ação e pausa na resposta do modelo
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE) # função regex para encontrar ferramenta e argumento
            chosen_tool = action[0][0] # armazena ferramenta solicitada pelo modelo 
            arg = action[0][1] # rmazena o argumento passado para essa ferramenta

            if chosen_tool in tools: # validação com a lista de ferramentas
                result_tool = eval(f"{chosen_tool}('{arg}')") # interpreta a string como código e executa-a
                next_prompt = f"Observation: {result_tool}" # informa ao modelo o resultado da ferramenta executada

            else:
                next_prompt = "Observation: Tool not found" # tratar erro caso não ncontre a ferramenta

            print(next_prompt) # registra no "log"
            continue

        if "Answer" in result: # consição para quebrar o loop, caso a resposta seja gerada
            break


loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?") # função de loop iniciada com a query

Thought: I need to find the masses of Earth and Saturn and then perform the required calculations.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I have the mass of Earth, now I need to get the mass of Saturn.

Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: I have the masses of Earth and Saturn. Now I need to add them together.

Action: calculate: 5.972e24 + 5.683e26
PAUSE
Observation: 5.74272e+26
Thought: I have the sum of the masses of Earth and Saturn. Now I need to multiply this by 2.

Action: calculate: 5.74272e26 * 2
PAUSE
Observation: 1.148544e+27
Thought: I have the final result.

Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.
